In [1]:
import datasets
acs_task_qa = datasets.load_dataset(
    path="acruz/folktexts",
    name="ACSIncome",   # Choose which task you want to load
    split="test")       # Choose split according to your intended use case

acs_task_qa

/scratch/gpfs/vv7118/projects/envs/folktexts/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['id', 'description', 'instruction', 'question', 'choices', 'answer', 'answer_key', 'choice_question_prompt', 'numeric_question', 'label', 'numeric_question_prompt', 'AGEP', 'COW', 'SCHL', 'MAR', 'OCCP', 'POBP', 'RELP', 'WKHP', 'SEX', 'RAC1P'],
    num_rows: 166450
})

In [1]:
from folktexts.acs import ACSDataset
acs_task_name = "ACSIncome"
dataset = ACSDataset.make_from_task(acs_task_name, cache_dir="/scratch/gpfs/vv7118/projects/finetuning-folktexts/data/")   # use `.subsample(0.01)` to get faster approximate results


/scratch/gpfs/vv7118/projects/envs/folktexts/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading ACS data...


In [5]:
x, y = dataset.get_train()
x['label'] = y

In [12]:
x

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,label
1080377,53,6.0,21.0,1,2752.0,36,0,20.0,1,1,0
2708379,51,1.0,14.0,1,8990.0,303,0,20.0,1,1,0
2706282,43,1.0,19.0,3,4840.0,26,0,40.0,1,9,1
1727779,63,1.0,17.0,1,6200.0,46,1,55.0,1,3,1
253929,48,2.0,24.0,1,230.0,6,1,60.0,2,6,1
...,...,...,...,...,...,...,...,...,...,...,...
2736277,54,1.0,22.0,1,5240.0,48,1,46.0,1,1,1
2778792,71,1.0,21.0,1,10.0,22,0,45.0,1,1,1
2588106,53,2.0,19.0,3,2360.0,46,13,40.0,2,1,0
217506,28,2.0,21.0,5,2350.0,6,10,30.0,2,1,0


In [11]:
cols = x.columns.tolist()[0:-1]
for i in range(len(cols)):
    cols_without_i = [k for k in cols if k != cols[i]]
    assert len(cols_without_i) == len(cols) - 1
    print(x.groupby(cols_without_i)['label'].mean())


COW  SCHL  MAR  OCCP    POBP  RELP  WKHP  SEX  RAC1P
1.0  1.0   1    10.0    1     1     45.0  1    1        1.0
                        6     1     50.0  1    1        1.0
                        17    0     45.0  1    1        1.0
                        36    1     55.0  2    1        1.0
                        42    1     50.0  1    1        1.0
                                                       ... 
8.0  24.0  5    3090.0  17    2     10.0  1    1        0.0
                        30    13    40.0  1    1        0.0
                        72    2     40.0  2    1        0.0
                4010.0  39    0     80.0  2    1        0.0
                4700.0  12    0     40.0  2    2        1.0
Name: label, Length: 1096248, dtype: float64
AGEP  SCHL  MAR  OCCP    POBP  RELP  WKHP  SEX  RAC1P
17    1.0   1    2360.0  207   15    10.0  2    6        0.0
                 4120.0  18    2     10.0  1    2        0.0
                 9620.0  48    0     40.0  1    1        1.0
     

In [3]:
from folktexts.acs import ACSTaskMetadata
from folktexts.acs.acs_thresholds import acs_income_threshold

# Instantiate folktables tasks
acs_income_task = ACSTaskMetadata.make_folktables_task(
    name="ACSIncome",
    target_threshold=acs_income_threshold,
    description="predict whether an individual's income is above $50,000",
)

ERROR:root:A task with `name='ACSIncome'` already exists. Overwriting...


Plan:

- take the original dataframe
- 

In [4]:
from folktexts.prompting import encode_row_prompt

print(encode_row_prompt(dataset.data.iloc[0], acs_income_task))


The following data corresponds to a survey respondent. The survey was conducted among US residents in 2018. Please answer the question based on the information provided. The data provided is enough to reach an approximate answer.

Information:
- The age is: 18 years old.
- The class of worker is: Working for a for-profit private company or organization.
- The highest educational attainment is: Some college, less than 1 year.
- The marital status is: Never married.
- The occupation is: Cashiers.
- The place of birth is: Georgia.
- The relationship to the reference person in the survey is: Non-institutionalized group quarters population.
- The usual number of hours worked per week is: 21 hours.
- The sex is: Female.
- The race is: Black or African American.

Question: What is this person's estimated yearly income?
A. Below $50,000.
B. Above $50,000.
Answer:


In [5]:
# Load transformers model
from folktexts.llm_utils import load_model_tokenizer
model, tokenizer = load_model_tokenizer("gpt2")   # using tiny model as an example

In [6]:
from folktexts.benchmark import Benchmark, BenchmarkConfig
bench = Benchmark.make_benchmark(
    task=acs_task_name, dataset=dataset,  # These vars are defined in the snippet above
    model=model, tokenizer=tokenizer,
    numeric_risk_prompting=True,    # See the full list of configs below in the README
)

Using zero-shot prompting.


In [13]:
bench.make_acs_benchmark('ACSIncome', model=model)

FileNotFoundError: [Errno 2] No such file or directory: '/home/vv7118/data/folktables'

In [6]:
X_test, y_test = dataset.get_test()


In [10]:


from folktexts.acs import ACSDataset
acs_task_name = "ACSIncome"     # Name of the benchmark ACS task to use

# Create an object that classifies data using an LLM
from folktexts import TransformersLLMClassifier
clf = TransformersLLMClassifier(
    model=model,
    tokenizer=tokenizer,
    task=acs_task_name,
)
# NOTE: You can also use a web-hosted model like GPT4 using the `WebAPILLMClassifier` class

# Use a dataset or feed in your own data
dataset = ACSDataset.make_from_task(acs_task_name, cache_dir="/scratch/gpfs/vv7118/projects/finetuning-folktexts/data/")   # use `.subsample(0.01)` to get faster approximate results

# You can compute risk score predictions using an sklearn-style interface
X_test, y_test = dataset.get_test()
test_scores = clf.predict_proba(X_test)

Loading ACS data...


Computing risk estimates:   2%|▏         | 165/10404 [01:58<2:02:08,  1.40it/s]


KeyboardInterrupt: 